In [1]:
from Define_Model.TDNN.TDNN import TDNN_v5
import kaldi_io
import torch

from Define_Model.Loss.SoftmaxLoss import AdditiveMarginLinear
import numpy as np
import kaldi_io
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
plt.rc('font', family='Times New Roman')

In [2]:
model = TDNN_v5(num_classes=1211, embedding_size=512, input_dim=40, alpha=0., input_norm='Mean',
                 encoder_type='STAP', activation='relu',
                 num_classes_b=0, block_type='basic', first_2d=False, stride=[1],
                 mask='None', mask_len=20, channels=[512, 512, 512, 512, 1500])

# model.classifier = AdditiveMarginLinear(feat_dim=512, num_classes=1211)
ckp = torch.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/checkpoint/TDNN_v5/vox1/klfb_egs_baseline/soft/featcombined_inputMean_STAP_em512_wde3_var/checkpoint_50.pth')
model.load_state_dict(ckp['state_dict'][0])

model.eval()

/home/yangwenhao/anaconda3/envs/py37/lib/python3.7/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.loss.CrossEntropyLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


TDNN_v5(
  (inst_layer): Mean_Norm(dim=-2)
  (frame1): TimeDelayLayer_v5(
    (kernel): Conv1d(40, 512, kernel_size=(5,), stride=(1,))
    (nonlinearity): ReLU(inplace=True)
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (frame2): TimeDelayLayer_v5(
    (kernel): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(2,))
    (nonlinearity): ReLU(inplace=True)
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (frame3): TimeDelayLayer_v5(
    (kernel): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(3,))
    (nonlinearity): ReLU(inplace=True)
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (frame4): TimeDelayLayer_v5(
    (kernel): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
    (nonlinearity): ReLU(inplace=True)
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (frame5): TimeDela

In [3]:
stmx = torch.nn.Softmax(dim=1)

In [17]:
x00 = kaldi_io.read_mat('/home/yangwenhao/local/project/lstm_speaker_verification/data/vox1/klfb/fbank/dev_fb40/raw_fbank_dev_fb40.1.ark:69000')
x0a = kaldi_io.read_mat('/home/yangwenhao/local/project/lstm_speaker_verification/data/vox1/klfb/fbank/dev_fb40/raw_fbank_dev_fb40.1.ark:26')
x0b = kaldi_io.read_mat('/home/yangwenhao/local/project/lstm_speaker_verification/data/vox1/klfb/fbank/dev_fb40/raw_fbank_dev_fb40.1.ark:33611')

x01 = kaldi_io.read_mat('/home/yangwenhao/local/project/lstm_speaker_verification/data/vox1/klfb/fbank/dev_fb40/raw_fbank_dev_fb40.1.ark:2686603')
x02 = kaldi_io.read_mat('/home/yangwenhao/local/project/lstm_speaker_verification/data/vox1/klfb/fbank/dev_fb40/raw_fbank_dev_fb40.1.ark:6070422')


x1 = kaldi_io.read_mat('/home/yangwenhao/local/project/lstm_speaker_verification/data/vox1/dev_aug_spk/fbank/raw_fbank_dev_aug_spk.1.ark:20')
x2 = kaldi_io.read_mat('/home/yangwenhao/local/project/lstm_speaker_verification/data/vox1/dev_aug_spk/fbank/raw_fbank_dev_aug_spk.1.ark:23267')


x00 = torch.FloatTensor(x00)[:,:40].unsqueeze(0)
x0a = torch.FloatTensor(x0a)[:,:40].unsqueeze(0)
x0b = torch.FloatTensor(x0b)[:,:40].unsqueeze(0)

print(x00.shape)
print(x0a.shape)
print(x0b.shape)


x01 = torch.FloatTensor(x01)[:,:40].unsqueeze(0)
x02 = torch.FloatTensor(x02)[:,:40].unsqueeze(0)
print(x01.shape)
print(x02.shape)



x1 = torch.FloatTensor(x1)[:,:40].unsqueeze(0)
x2 = torch.FloatTensor(x2)[:,:40].unsqueeze(0)
print(x1.shape)
print(x2.shape)

torch.Size([1, 558, 40])
torch.Size([1, 810, 40])
torch.Size([1, 854, 40])
torch.Size([1, 414, 40])
torch.Size([1, 434, 40])
torch.Size([1, 558, 40])
torch.Size([1, 558, 40])


In [18]:
logits0a, embedding_b0a = model(x0a)
logits0b, embedding_b0b = model(x0b)

logits00, embedding_b00 = model(x00)
logits01, embedding_b01 = model(x01)
logits02, embedding_b02 = model(x02)

logits1, embedding_b1 = model(x1)
logits2, embedding_b2 = model(x2)

In [19]:
print(stmx(logits00)[0].sort())
print(stmx(logits0a)[0].sort())
print(stmx(logits0b)[0].sort())

torch.return_types.sort(
values=tensor([9.3096e-08, 1.3845e-07, 2.0548e-07,  ..., 1.4585e-01, 1.4924e-01,
        1.6458e-01], grad_fn=<SortBackward>),
indices=tensor([ 674,  676, 1139,  ...,  654,    0,  518]))
torch.return_types.sort(
values=tensor([3.0698e-07, 3.9696e-07, 6.3086e-07,  ..., 5.8833e-02, 6.9238e-02,
        3.4801e-01], grad_fn=<SortBackward>),
indices=tensor([110, 202, 688,  ..., 654, 543,   0]))
torch.return_types.sort(
values=tensor([1.5068e-07, 1.5888e-07, 1.6122e-07,  ..., 6.7626e-02, 1.3982e-01,
        3.1865e-01], grad_fn=<SortBackward>),
indices=tensor([110, 676, 193,  ..., 543, 518,   0]))


In [20]:
print(stmx(logits01)[0].sort())
print(stmx(logits02)[0].sort())


torch.return_types.sort(
values=tensor([2.6092e-07, 3.1600e-07, 4.3541e-07,  ..., 4.0313e-02, 6.7129e-02,
        3.4266e-01], grad_fn=<SortBackward>),
indices=tensor([1140,  294,  657,  ...,  313,  994,    1]))
torch.return_types.sort(
values=tensor([1.8703e-06, 1.8918e-06, 1.9672e-06,  ..., 4.2330e-02, 4.4083e-02,
        2.0894e-01], grad_fn=<SortBackward>),
indices=tensor([ 398, 1063,  377,  ..., 1074,  260,  928]))
torch.return_types.sort(
values=tensor([7.3378e-08, 1.1264e-07, 1.2866e-07,  ..., 1.0503e-01, 1.1848e-01,
        2.5894e-01], grad_fn=<SortBackward>),
indices=tensor([ 676,  674, 1139,  ...,  405,  654,  618]))


In [21]:
print(stmx(logits1)[0].sort())
print(stmx(logits2)[0].sort())

torch.return_types.sort(
values=tensor([7.3378e-08, 1.1264e-07, 1.2866e-07,  ..., 1.0503e-01, 1.1848e-01,
        2.5894e-01], grad_fn=<SortBackward>),
indices=tensor([ 676,  674, 1139,  ...,  405,  654,  618]))
torch.return_types.sort(
values=tensor([8.8413e-07, 1.2508e-06, 1.4454e-06,  ..., 6.4731e-02, 7.1566e-02,
        1.1731e-01], grad_fn=<SortBackward>),
indices=tensor([ 41,  91, 110,  ..., 654, 673,   0]))


torch.return_types.sort(
values=tensor([7.3378e-08, 1.1264e-07, 1.2866e-07,  ..., 1.0503e-01, 1.1848e-01,
        2.5894e-01], grad_fn=<SortBackward>),
indices=tensor([ 676,  674, 1139,  ...,  405,  654,  618]))

torch.return_types.sort(
values=tensor([8.8413e-07, 1.2508e-06, 1.4454e-06,  ..., 6.4731e-02, 7.1566e-02,
        1.1731e-01], grad_fn=<SortBackward>),
indices=tensor([ 41,  91, 110,  ..., 654, 673,   0]))

In [23]:
speakers= []
with open('/home/yangwenhao/local/project/lstm_speaker_verification/data/vox1/egs/klfb/dev_fb80/spk2utt', 'r') as f:
    for l in f.readlines():
        speakers.append(l.split()[0])
speakers.sort()

spk_to_idx = {speakers[i]: i for i in range(len(speakers))}

In [24]:
spk_to_idx

{'id10001': 0,
 'id10002': 1,
 'id10003': 2,
 'id10004': 3,
 'id10005': 4,
 'id10006': 5,
 'id10007': 6,
 'id10008': 7,
 'id10009': 8,
 'id10010': 9,
 'id10011': 10,
 'id10012': 11,
 'id10013': 12,
 'id10014': 13,
 'id10015': 14,
 'id10016': 15,
 'id10017': 16,
 'id10018': 17,
 'id10019': 18,
 'id10020': 19,
 'id10021': 20,
 'id10022': 21,
 'id10023': 22,
 'id10024': 23,
 'id10025': 24,
 'id10026': 25,
 'id10027': 26,
 'id10028': 27,
 'id10029': 28,
 'id10030': 29,
 'id10031': 30,
 'id10032': 31,
 'id10033': 32,
 'id10034': 33,
 'id10035': 34,
 'id10036': 35,
 'id10037': 36,
 'id10038': 37,
 'id10039': 38,
 'id10040': 39,
 'id10041': 40,
 'id10042': 41,
 'id10043': 42,
 'id10044': 43,
 'id10045': 44,
 'id10046': 45,
 'id10047': 46,
 'id10048': 47,
 'id10049': 48,
 'id10050': 49,
 'id10051': 50,
 'id10052': 51,
 'id10053': 52,
 'id10054': 53,
 'id10055': 54,
 'id10056': 55,
 'id10057': 56,
 'id10058': 57,
 'id10059': 58,
 'id10060': 59,
 'id10061': 60,
 'id10062': 61,
 'id10063': 62,
 '